<a href="https://colab.research.google.com/github/yuooka/kaggleops-tutorial/blob/master/run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

# 各種設定

In [ ]:
import os

from google.colab import auth
auth.authenticate_user()

PROJECT_ID = 'kaggleops-project'
BUCKET_NAME = 'kaggleops-bucket'

os.environ["GCLOUD_PROJECT"] = PROJECT_ID

In [ ]:
!git clone https://github.com/yuooka/kaggleops-tutorial.git

In [ ]:
!pip install -qr ./kaggleops-tutorial/requirements.txt

In [ ]:
cd ./kaggleops-tutorial/src

In [ ]:
!gcloud config set project {PROJECT_ID}

In [ ]:
!gsutil -mq cp -rn gs://{BUCKET_NAME}/mlruns /content/{BUCKET_NAME}/src

# プログラム実行

In [ ]:
!python mlproject.py

# ローカルに保存されたmlrunsをGCSに転送

In [ ]:
!gsutil -mq cp -rn /content/{BUCKET_NAME}/src/mlruns gs://{BUCKET_NAME}